# METODOLOGIA DA ANÁLISE DE ABSENTISMO
**Documento Técnico de Decisões e Procedimentos**

---

## 1. CONTEXTO E OBJETIVOS

### 1.1 Dataset
- **Registos:** ~1.3M
- **Colaboradores:** 3,135
- **Período:** 18 meses (Janeiro 2024 - Junho 2025)
- **Tipo:** Call Center

### 1.2 Objetivo Geral
Análise descritiva e prescritiva de padrões de absentismo para identificar:
- Taxas de absentismo e atrasos
- Padrões comportamentais disruptivos
- Colaboradores e segmentos de risco
- Tendências temporais
- Recomendações acionáveis para a direção

---

## 2. PREPARAÇÃO DE DADOS (GRUPO 1)

### 2.1 Classificação de Códigos
**Decisão:** Aplicar nova classificação hierárquica (Códigos V2)
- **Nivel 1:** Categoria principal (ex: Trabalho Pago, Falta Justificada, Atraso)
- **Nivel 2:** Subcategoria (ex: Ausência Médica, Falta Injustificada)

### 2.2 Identificação de Incompatibilidades
**Metodologia:**
- Criar matriz de compatibilidade entre categorias de Nivel 2
- Identificar dias com múltiplos registos incompatíveis
- **Exemplos de incompatibilidades:**
  - Presença + Ausência Médica
  - Trabalho Pago + Falta Injustificada

**Decisão:** Remover dias incompatíveis do dataset (exportados para revisão)

### 2.3 Normalização de Categorias Profissionais
**Problema:** Múltiplas grafias para mesma categoria
**Solução:** Mapeamento manual para categorias padronizadas
**Timing:** ANTES da separação em dataframes hierárquicos

### 2.4 Separação com Hierarquias
**Decisão Crítica:** Criar dois dataframes separados:

#### DataFrame 1: `df` (Absentismo)
- **1 linha por dia por colaborador**
- **Hierarquia de prioridade:**
  1. Trabalho Pago (prioritário)
  2. Falta Justificada
  3. Falta Injustificada
  4. Outras categorias
- **Exclusão:** Atrasos (tratados separadamente)
- **Uso:** Cálculo de taxas de absentismo

#### DataFrame 2: `df_atrasos` (Atrasos)
- **1 linha por dia por colaborador**
- **Hierarquia de prioridade:**
  1. Atraso (prioritário sobre tudo)
  2. Trabalho Pago
  3. Outras categorias
- **Uso:** Análise específica de atrasos

**Justificação:** Um colaborador pode trabalhar (Trabalho Pago) E ter atraso no mesmo dia. A separação evita contagem dupla.

### 2.5 Validação Final
**Checks aplicados:**
1. Sem dias duplicados em cada dataframe
2. Sem atrasos em `df` (absentismo)
3. Atrasos presentes apenas em `df_atrasos`

---

## 3. MÉTRICAS E DATAFRAMES DERIVADOS (GRUPO 2)

### 3.1 Dataframes Criados

#### `df_faltas`
```python
df_faltas = df[df['Nivel 1'].isin(['Falta Justificada', 'Falta Injustificada'])]
```
- **Uso:** Numerador para taxas de absentismo
- **Contém:** Apenas ausências que contam como absentismo

#### `df_base_absentismo`
```python
df_base_absentismo = df[df['Nivel 1'].isin([
    'Trabalho Pago', 'Falta Justificada', 'Falta Injustificada'
])]
```
- **Uso:** Denominador para taxas de absentismo
- **Contém:** Dias em que o colaborador estava escalado (presença + faltas)
- **Exclui:** Férias, licenças, formação (ausências planeadas)

### 3.2 Fórmulas de Taxas

#### Taxa de Absentismo
```
Taxa = (Faltas / Base) × 100
```
Onde:
- **Faltas** = Falta Justificada + Falta Injustificada
- **Base** = Trabalho Pago + Faltas

#### Taxa de Atrasos
```
Taxa = (Atrasos / Trabalho Pago) × 100
```
Onde:
- **Atrasos** = Registos com Nivel 1 = 'Atraso' (de `df_atrasos`)
- **Trabalho Pago** = Registos com Nivel 1 = 'Trabalho Pago' (de `df_base_absentismo`)

### 3.3 Normalização em Análises Temporais
**Decisão Crítica:** Ao analisar padrões por dia da semana ou período do mês:
- **Sempre usar taxas normalizadas** (não valores absolutos)
- **Razão:** Segunda-feira tem naturalmente mais registos que sábado
- **Fórmula:** Taxa = (Eventos / Base) × escala (ex: 100, 1000)

### 3.4 Tratamento de Dados com Múltiplos Anos
**Problema:** Dataset tem 18 meses (2024-01 a 2025-06)
- Jan-Jun aparecem 2x (2024 e 2025)
- Jul-Dez aparecem 1x (apenas 2024)

**Decisão:**
- **Para médias mensais:** Calcular média de cada mês considerando múltiplas ocorrências
- **Para comparações anuais:** Usar apenas meses equivalentes (Jan-Jun 2024 vs Jan-Jun 2025)

---

## 4. ANÁLISE DE SPELLS (GRUPO 3)

### 4.1 Definição de Spell
**Spell** = Episódio contínuo de ausência (um ou mais dias consecutivos)

**Algoritmo:**
1. Ordenar `df_faltas` por colaborador e data
2. Calcular diferença de dias entre registos consecutivos
3. Novo spell quando:
   - É o primeiro registo do colaborador, OU
   - Gap > 1 dia (não consecutivo)

### 4.2 Categorização de Spells
- **1 dia:** Ausência pontual
- **2-3 dias:** Short-term
- **4-7 dias:** Médio prazo
- **8-14 dias:** Longo prazo
- **>14 dias:** Muito longo (doença grave/recuperação)

**Flags especiais:**
- `short_term`: duracao_dias ≤ 3
- `long_term`: duracao_dias > 14

### 4.3 DataFrame de Spells
**Criado:** `df_spells`
**Contém:**
- Identificador único do spell
- Login e nome do colaborador
- Data início, data fim, duração
- Tipo predominante (Nivel 1 e Nivel 2)
- Dia da semana de início e fim
- Mês e ano

### 4.4 Análise por Colaborador
**Criado:** `df_colab_spells`
**Métricas por colaborador:**
- Número de spells (frequency rate)
- Total de dias ausentes
- Duração média, mediana, desvio padrão
- Número de spells short-term
- Número de spells long-term

---

## 5. BRADFORD FACTOR ANALYSIS (GRUPO 5)

### 5.1 Decisões Críticas de Escopo

#### Período de Análise
**Decisão:** Apenas 2025 (não período total)
**Justificação:**
- Bradford é ferramenta de gestão presente/futura
- Colaboradores inativos não podem ser alvo de intervenção
- Dados mais recentes são mais relevantes

#### População Analisada
**Decisão:** Apenas colaboradores ativos em 2025
**Filtro:** `Activo == 'Sim'`
**Justificação:**
- Foco em quem está atualmente na empresa
- Permite ação imediata sobre casos identificados

#### Tipo de Ausências
**Decisão:** Apenas spells SHORT-TERM (≤3 dias)
**Justificação:**
1. **Foco em padrões disruptivos:** Faltas curtas e frequentes são mais problemáticas operacionalmente
2. **Distinguir doença de comportamento:** Spells longos (ex: 20 dias) geralmente são ausências médicas legítimas
3. **Incluir TODAS as short-term:** Mesmo baixas médicas curtas (podem ser abusadas)

**Nome:** "Bradford Disruptivo" (para deixar claro o foco)

### 5.2 Fórmula e Thresholds

#### Fórmula
```
Bradford Disruptivo = S² × D
```
Onde:
- **S** = Número de spells short-term (≤3 dias) em 2025
- **D** = Total de dias desses spells

#### Thresholds Aplicados
| Score | Categoria | Ação Recomendada |
|-------|-----------|------------------|
| < 50 | Aceitável | Monitorização normal |
| 50-100 | Monitorizar | Prestar atenção |
| 100-200 | Atenção | Conversa informal |
| 200-400 | Alto Risco | Reunião formal |
| 400-600 | Risco Severo | Aviso formal |
| > 600 | Risco Crítico | Ação disciplinar |

### 5.3 Análise por Segmentos
**Filtros aplicados:**
- Operações: ≥ 20 colaboradores
- Categorias Profissionais: ≥ 20 colaboradores

**Justificação:** Amos (muitos funcionários trabalhao ao fim de semana)tras pequenas (<20) têm médias enviesadas

---

## 6. DETECÇÃO DE PADRÕES SUSPEITOS (GRUPO 6)

### 6.1 Padrão Segunda/Sexta (Weekend Pattern)

#### Metodologia
- Contar spells short-term que começam à segunda OU terminam à sexta
- Calcular proporção por colaborador

#### Flag de Suspeita
**Critério:**
- > 50% dos spells começam à segunda OU terminam à sexta
- E mínimo de 5 spells (significância estatística)

#### Baseline Esperado
- 20% (distribuição uniforme em 5 dias úteis)
- Desvios significativos indicam possível padrão

### 6.2 Padrão de Ponte (Bridge Pattern)

#### Metodologia
- Definir lista de feriados nacionais (Angola)
- Identificar spells que começam/terminam ±1 dia de feriado

#### Flag de Suspeita
**Critério:**
- > 40% dos spells são adjacentes a feriados
- E mínimo de 5 spells

### 6.3 Baixas Médicas Short-Term

#### Metodologia
- Filtrar spells short-term com `Nivel 2 == 'Ausência Médica'`
- Analisar distribuição por dia da semana

#### Flag de Suspeita
**Critério:**
- ≥ 3 baixas médicas short-term
- > 60% começam à segunda-feira

**Interpretação:** Concentração anormal em segundas pode indicar abuso

### 6.4 Síntese Multi-Flag
**Criado:** Score de suspeita por colaborador
- 0 flags: Sem padrões suspeitos
- 1 flag: Atenção ligeira
- 2 flags: Alta suspeita
- 3 flags: Muito suspeito (investigação prioritária)

---

## 7. ANÁLISE DE COHORTS (GRUPO 7)

### 7.1 Definição de Cohorts
**Base:** Campo `DtActivacao` (data de entrada na empresa)
**Referência:** 30 de Junho de 2025

#### Categorias
- **<1 ano:** Colaboradores novos
- **1-2 anos:** Juniores
- **2-3 anos:** Intermediários
- **3-5 anos:** Seniores
- **>5 anos:** Muito seniores

### 7.2 Métricas por Cohort
- Taxa de absentismo
- Média de dias de falta por colaborador
- Distribuição de spells

### 7.3 Hipóteses Testadas
1. Novos colaboradores têm mais faltas? (período de adaptação)
2. Ou menos faltas? (honeymoon period, medo de consequências)

**Decisão:** Análise apenas 2025, colaboradores ativos

---

## 8. CLUSTERING (GRUPO 8)

### 8.1 Features Utilizadas
```python
features = [
    'num_spells_short',      # Frequência
    'total_dias_short',      # Volume
    'bradford_disruptivo'    # Score composto
]
```

### 8.2 Algoritmo
**K-Means:**
- Normalização: StandardScaler (essencial para K-Means)
- Determinação de K:
  - Elbow Method (inércia)
  - Silhouette Score (qualidade da separação)
- K escolhido: Melhor Silhouette Score

### 8.3 Interpretação de Clusters
**Perfis típicos:**
- Cluster "Baixo Risco": Poucos spells, Bradford < 50
- Cluster "Moderado": Alguns spells, Bradford 50-150
- Cluster "Alto Risco": Muitos spells, Bradford 150-300
- Cluster "Crítico": Spells frequentes, Bradford > 300

**Uso:** Segmentação para ações diferenciadas

---

## 9. ANÁLISE DE REDES (GRUPO 9)

### 9.1 Conceito
**Co-ausência:** Quando dois colaboradores da mesma operação faltam no mesmo dia

### 9.2 Metodologia
1. Agrupar faltas por (Data, Operação)
2. Criar pares de colaboradores que faltaram juntos
3. Contar frequência de cada par
4. Filtrar pares com ≥ 3 co-ausências (significativo)

### 9.3 Visualização
**Network Graph:**
- Nós = Colaboradores
- Arestas = Co-ausências
- Espessura = Frequência de co-ausência

### 9.4 Interpretação
**Possíveis causas:**
- Contágio social (influência de pares)
- Turnos/equipas fixas
- Problemas comuns (transporte, gestão)

---

## 10. COMPARAÇÃO ANUAL (GRUPO 9B)

### 10.1 Período de Comparação
**Decisão Crítica:** Apenas Jan-Jun de cada ano
**Justificação:**
- 2024: Jan-Dez disponível
- 2025: Jan-Jun disponível
- **Comparação justa:** Apenas meses equivalentes

### 10.2 Métricas Comparadas
- Taxa de absentismo
- Número de spells
- Duração média de spells
- Distribuição de spells por duração

### 10.3 Análise de Tendência
**Método:** Comparar médias dos períodos equivalentes
**Resultado:** Variação percentual 2024 → 2025

---

## 11. CONTROL CHARTS (GRUPO 10)

### 11.1 U-Chart
**Tipo:** Control chart para dados de contagem (taxa por unidade)

### 11.2 Metodologia
**Dados:**
- Taxa semanal de faltas por colaborador (2025)
- Base: colaboradores ativos (Trabalho Pago) por semana

**Limites de Controle:**
```
UCL = média + 3 × √(média / n)
LCL = média - 3 × √(média / n)
```
Onde n = número de colaboradores ativos

### 11.3 Interpretação
**Semanas fora de controle:**
- Acima UCL: Pico anormal de ausências (investigar)
- Abaixo LCL: Ausências anormalmente baixas

**Ação:** Investigar causas especiais em semanas fora de controle

---

## 12. VISUALIZAÇÕES (GRUPO 11)

### 12.1 Calendar Heatmap
**Objetivo:** Identificar dias/semanas críticos visualmente

**Estrutura:**
- Eixo X: Dias da semana
- Eixo Y: Semanas do ano
- Cor: Intensidade de faltas

**Uso:** Identificar padrões sazonais ou eventos específicos

---

## 13. DECISÕES TRANSVERSAIS

### 13.1 Tratamento de Outliers
**Abordagem:** NÃO remover, mas identificar e contextualizar
**Justificação:** Outliers podem ser casos críticos que requerem atenção

### 13.2 Visualizações
**Princípios:**
- Profissionais (sem emojis)
- Títulos descritivos com contexto
- Sempre incluir baseline/referência quando relevante
- Cores consistentes ao longo da análise

### 13.3 Agregação vs Individualização
**Decisão:** Focar em métricas agregadas
**Justificação:**
- Objetivo é apresentação à direção (insights gerais)
- Listas individuais pouco úteis (colaboradores podem já não estar na empresa)
- Foco em segmentos acionáveis (operações, categorias)

### 13.4 Filtros de Amostra Mínima
**Regra:** Operações/Categorias com < 20 colaboradores são excluídas de rankings
**Justificação:** Médias de amostras pequenas são enviesadas e não representativas

---

## 14. ORDEM DE EXECUÇÃO

A análise DEVE ser executada nesta ordem:

1. **GRUPO 1:** Preparação e limpeza
2. **GRUPO 2:** Métricas fundamentais
3. **GRUPO 3:** Análise de spells
4. **GRUPO 4:** KPIs e atrasos
5. **GRUPO 5:** Bradford Factor
6. **GRUPO 6:** Padrões suspeitos
7. **GRUPO 7:** Cohorts
8. **GRUPO 8:** Clustering
9. **GRUPO 9:** Redes sociais
10. **GRUPO 10:** Comparação anual
11. **GRUPO 11:** Control charts
12. **GRUPO 12:** Visualizações finais

**Dependências Críticas:**
- Grupo 3 depende do Grupo 1 (df_faltas)
- Grupo 5 depende do Grupo 3 (df_spells, df_colab_spells)
- Grupo 6 depende do Grupo 5 (df_bradford_disruptivo)

---

## 15. LIMITAÇÕES E NOTAS

### 15.1 Limitações Conhecidas
1. **Causa raiz:** Análise identifica padrões, não causas
2. **Contexto individual:** Métricas agregadas não capturam situações pessoais
3. **Dados históricos:** Alguns colaboradores podem já não estar na empresa

### 15.2 Recomendações para Uso
1. **Bradford Factor:** Usar como trigger para conversa, não punição automática
2. **Padrões suspeitos:** Investigar contexto antes de assumir má conduta
3. **Segmentos críticos:** Priorizar intervenções em grupos de alto impacto

### 15.3 Próximos Passos Sugeridos
1. Validação de padrões suspeitos com gestores
2. Entrevistas com colaboradores em risco crítico
3. Análise de causas raiz (qualitativa)
4. Planos de ação diferenciados por segmento

---

## RESUMO EXECUTIVO DE DECISÕES CHAVE

| Decisão | Justificação | Impacto |
|---------|--------------|---------|
| 2 dataframes separados (absentismo + atrasos) | Evitar contagem dupla | Taxas corretas |
| Bradford: apenas 2025, ativos, short-term | Foco operacional presente | Ações imediatas |
| Normalização por base em análises temporais | Ajustar por volume variável | Comparações justas |
| Comparações anuais: Jan-Jun apenas | Meses equivalentes | Comparação válida |
| Filtro ≥20 colaboradores em rankings | Evitar viés de amostras pequenas | Insights robustos |
| Foco em agregados, não listas individuais | Objetivo de apresentação | Clareza executiva |

---

**Documento criado:** 10 Novembro 2025
**Versão:** 1.0
**Autores:** Análise de Absentismo - Call Center